## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-17-16-52-01 +0000,nyt,Trump Administration Live Updates: European Le...,https://www.nytimes.com/live/2025/08/17/us/tru...
1,2025-08-17-16-46-20 +0000,nyt,U.S. Pauses Visitor Visas for Gazans After Lau...,https://www.nytimes.com/2025/08/16/us/politics...
2,2025-08-17-16-36-49 +0000,nyt,Trump’s Selective Stance on Justice: Redemptio...,https://www.nytimes.com/2025/08/17/us/politics...
3,2025-08-17-16-31-35 +0000,nyt,Trump and Putin Find Common Ground on One Issu...,https://www.nytimes.com/2025/08/17/us/politics...
4,2025-08-17-16-30-00 +0000,nypost,Want to beat the next big Coinbase listing? He...,https://nypost.com/business/next-big-crypto-co...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2319/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,35
24,putin,21
69,ukraine,17
103,strike,11
106,war,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
157,2025-08-16-18-48-22 +0000,nypost,How Trump’s meeting with Putin pivoted from Uk...,https://nypost.com/2025/08/16/us-news/inside-h...,106
108,2025-08-17-01-10-58 +0000,nypost,First Lady Melania Trump calls for end to war ...,https://nypost.com/2025/08/16/world-news/first...,105
135,2025-08-16-21-22-09 +0000,nyt,Trump Backs Off Cease-Fire Demand in Ukraine W...,https://www.nytimes.com/2025/08/16/world/europ...,103
153,2025-08-16-19-25-47 +0000,wapo,Russia sees victory as Trump adopts Putin’s ap...,https://www.washingtonpost.com/world/2025/08/1...,98
156,2025-08-16-18-51-12 +0000,nyt,Trump Bows to Putin’s Approach on Ukraine: No ...,https://www.nytimes.com/2025/08/16/us/politics...,95


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
157,106,2025-08-16-18-48-22 +0000,nypost,How Trump’s meeting with Putin pivoted from Uk...,https://nypost.com/2025/08/16/us-news/inside-h...
150,49,2025-08-16-19-41-53 +0000,nypost,Trump’s efforts at Alaska summit applauded by ...,https://nypost.com/2025/08/16/world-news/trump...
142,48,2025-08-16-20-15-27 +0000,wapo,Canada orders Air Canada flight attendants bac...,https://www.washingtonpost.com/world/2025/08/1...
44,40,2025-08-17-12-18-22 +0000,nypost,"European leaders, NATO boss will back up Zelen...",https://nypost.com/2025/08/17/world-news/europ...
13,33,2025-08-17-15-41-00 +0000,wsj,Israelis Hold Nationwide Protests and Strike t...,https://www.wsj.com/world/middle-east/israelis...
17,29,2025-08-17-15-34-03 +0000,nypost,More than 700 National Guard troops from 3 GOP...,https://nypost.com/2025/08/17/us-news/more-tha...
5,27,2025-08-17-16-22-13 +0000,nypost,LI teen allegedly attacked by rival’s mom with...,https://nypost.com/2025/08/17/us-news/long-isl...
86,25,2025-08-17-07-05-52 +0000,nypost,Public sends flood of donations to inmate who ...,https://nypost.com/2025/08/17/us-news/californ...
48,24,2025-08-17-11-36-18 +0000,nypost,Minnesota fast food worker Rodolfo Depaz who w...,https://nypost.com/2025/08/17/us-news/minnesot...
27,23,2025-08-17-14-21-19 +0000,nypost,Disgraced ex-Rep. Anthony Weiner makes blunt p...,https://nypost.com/2025/08/17/us-news/disgrace...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
